In [1]:
import sys 
sys.path.append('..')

In [2]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import dlib
import os, pickle
import re
import signal
import operator
from sklearn import svm
from sklearn import tree

## Rearrange Embeddings for training

Rearrange data to be compatible for training with sklearn

In [3]:
def rearrange_people_reps(people_reps):
    all_reps = []
    classes = []
    for person in range(len(people_reps)):
        for rep in range(len(people_reps[person])):
            all_reps.append(people_reps[person][rep])
            classes.append(person)
    return all_reps, classes            

## Load embeddings

Load all Representation of own dataset (To be Created with Facenet_Embeddings.ipyn)

In [4]:
names = np.load('../models/facenet_own_names.npy')
people_reps = np.load('../models/facenet_own_reps.npy')

Reshape for training

In [5]:
reps, classes = rearrange_people_reps(people_reps)

## Train SVM and save to file

In [6]:
clf = svm.SVC(probability=True)
clf.fit(reps, classes) 

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [7]:
save_dir = '../models/SVM/'
os.makedirs(os.path.dirname(save_dir), exist_ok=True)
pickle.dump( clf, open( save_dir + 'own.b' , "wb" ) )

## Train Decision Tree and save to file

In [8]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(reps, classes)

In [9]:
save_dir = '../models/Tree/'
os.makedirs(os.path.dirname(save_dir), exist_ok=True)
pickle.dump( clf, open( save_dir + 'own.b' , "wb" ) )

--> TODO: Try Tensorflow Model (e.g. 2 fully connected layers) to map from embeddings to classes